## 필요한 모듈 모두 불러오기

In [1]:
import os, re, glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

## 정규 표현식으로 데이터 정제하기

In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


## 데이터 불러오기

In [3]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 정제된 문장 정리하기

In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## 정제된 문장 토큰화 하기

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다 (15개 제한)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15    1 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4f53dfef70>


## 학습시킬 문장과 타겟 문장 지정하기

In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

## 훈련데이터와 평가데이터 분리하기

In [7]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)  

Source Train: (140599, 14)
Target Train: (140599, 14)


## 인공지능 만들기(텍스트 제네레이터 모델 설계해서 변수에 담아주기)

In [17]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048 # 예문에서는 1024 데이터 양이 늘었으므로 증가 시켜 보았다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

## 모델 학습시키기

In [18]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size = 256, validation_data=(enc_val, dec_val))

Epoch 1/10
550/550 [==============================] - 256s 432ms/step - loss: 3.4888 - val_loss: 3.0593
Epoch 2/10
550/550 [==============================] - 238s 432ms/step - loss: 2.9204 - val_loss: 2.8090
Epoch 3/10
550/550 [==============================] - 238s 433ms/step - loss: 2.6716 - val_loss: 2.6494
Epoch 4/10
550/550 [==============================] - 238s 432ms/step - loss: 2.4388 - val_loss: 2.5244
Epoch 5/10
550/550 [==============================] - 238s 432ms/step - loss: 2.2030 - val_loss: 2.4249
Epoch 6/10
550/550 [==============================] - 238s 433ms/step - loss: 1.9648 - val_loss: 2.3398
Epoch 7/10
550/550 [==============================] - 238s 432ms/step - loss: 1.7327 - val_loss: 2.2766
Epoch 8/10
550/550 [==============================] - 238s 433ms/step - loss: 1.5176 - val_loss: 2.2367
Epoch 9/10
550/550 [==============================] - 238s 434ms/step - loss: 1.3298 - val_loss: 2.2183
Epoch 10/10
550/550 [==============================] - 238s 433m

## 학습 시킨 모델로 문장 출력하기!

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

## 모델이 출력한 문장!

In [26]:
# generate_text(model, tokenizer, init_sentence="<start> he")
test_sen = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
print(test_sen)

<start> i love you , baby <end> 


# 회고
이번에는 처음으로 자연어를 다루는 인공지능에 대해서 배웠는데 개념적으로는 어느정도 이해할 수 있었지만 워낙 생소한 분야라  
완벽하게 이해 하지는 못했다. LMS도 그부분을 인지 하고 있는것인지 간단하게 설명 해주시고
실습 부분도 모두 했던 코드를 그대로 사용해서 진행 할 수 있게끔 만들어 두어서 실습 자체에는 큰 어려움이 없었다. 다만 모델의
>Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계  

부분은 좀처럼 해결되지 않아  
https://goodtogreate.tistory.com/entry/Batch-%ED%81%AC%EA%B8%B0%EC%9D%98-%EA%B2%B0%EC%A0%95-%EB%B0%A9%EB%B2%95 에 적혀있는 mini-batch size 를 이용하니 어느정도 목표하던 값으로 향할 수 있었다.  
시각화 관련해서 매우 간단하게 정리해준 글이 있어서  

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=qbxlvnf11&logNo=221506748164
시각화를 이번에야 말로 적용 하려고 했었는데 학습 시간이 너무 오래 걸려서 이번까지만 미루기로 했다 ㅠㅠ

## 의문
<start>와 <end>로 학습문장과 타겟문장을 나누는 부분이 잘 이해 되지 않았다. 찾아서 볼 필요가 있을듯 하다!  


## 시행착오
Embedding Size와 Hidden Size를 조절하면 문장이 이상하게 나오는 경우도 있고 너무 오래 걸려서 도저희 진행할 수 없는 경우가 있어서  
사실 별로 좋은 시도는 아니였던것 같은데, 다른 사람들이 어떻게 해결했는지 나와 무엇이 다른지 확인해 볼 필요가 있다.
